In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-07"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13143,2024-03-07,Eua Nba,00:00,Golden State Warriors,Milwaukee Bucks,1.66,2.31,226.5,1.85,2.09,-3.5,2.01,2.90,zuSHSKh4,0.602410,0.432900,0.540541,0.478469,0.035310,0.2,0.0,NaN,NaN,208.528,62.325247,0.298882,187.364,57.180106,0.305182,194.250,201.392,203.94,175.15,0.0,0.0,0.0,0.0,0.231546,0.086145,0.256344,-0.43,-0.086,-7.674419,0.600925,0.5,-0.100925,0.46,0.092,14.239130,0.633183,0.5,-0.133183
13144,2024-03-07,Eua Nba,00:30,Los Angeles Lakers,Sacramento Kings,1.72,2.20,238.5,1.85,2.09,-2.5,2.01,2.55,r5VPQbNG,0.581395,0.454545,0.540541,0.478469,0.035941,0.4,0.8,NaN,NaN,250.116,62.207640,0.248715,234.420,80.350720,0.342764,235.136,291.046,238.96,280.50,0.0,0.0,0.0,0.0,0.173169,0.086145,0.167473,2.06,0.412,1.747573,0.595282,0.7,0.104718,-0.14,-0.028,-42.857143,0.456626,0.4,-0.056626
13145,2024-03-07,Eua Nba,21:00,Detroit Pistons,Brooklyn Nets,2.20,1.72,216.5,1.86,2.07,2.5,1.97,1.49,QTKUPIxN,0.454545,0.581395,0.537634,0.483092,0.035941,0.2,0.4,NaN,NaN,643.046,205.320670,0.319294,288.276,166.693697,0.578243,405.696,358.148,323.73,214.23,0.0,0.0,0.0,0.0,0.173169,0.075569,0.196191,3.54,0.708,1.694915,0.298988,0.2,-0.098988,-0.71,-0.142,-5.070423,0.384993,0.3,-0.084993
13146,2024-03-07,Eua Nba,21:00,Indiana Pacers,Minnesota Timberwolves,1.93,1.74,229.5,1.81,2.00,1.5,1.83,1.68,4KJYOxhT,0.518135,0.574713,0.552486,0.500000,0.092847,0.4,0.8,NaN,NaN,198.270,64.994545,0.327808,194.132,68.270141,0.351669,219.660,214.376,250.04,154.88,0.0,0.0,0.0,0.0,0.073215,0.070525,0.060436,2.28,0.456,2.039474,0.577780,0.7,0.122220,2.20,0.440,1.681818,0.679713,0.8,0.120287
13147,2024-03-07,Eua Nba,21:30,Dallas Mavericks,Miami Heat,1.39,2.69,227.5,1.82,1.99,-4.5,1.82,3.63,z360Kd0p,0.719424,0.371747,0.549451,0.502513,0.091172,0.4,0.2,NaN,NaN,207.360,50.847669,0.245214,338.942,100.958612,0.297864,210.286,303.894,206.64,415.03,0.0,0.0,0.0,0.0,0.450607,0.063101,0.469675,0.87,0.174,2.241379,0.578910,0.7,0.121090,8.12,1.624,1.040640,0.418038,0.7,0.281962
13148,2024-03-07,Europa Euroliga,14:30,Anadolu Efes,Bayern,1.51,2.64,165.5,1.91,1.91,-4.5,1.91,3.75,EytZCRf4,0.662252,0.378788,0.523560,0.523560,0.041040,0.6,0.4,NaN,NaN,146.110,29.206436,0.199893,163.682,81.242296,0.496342,154.860,237.832,165.90,286.77,0.0,0.0,0.0,0.0,0.385075,0.000000,0.459744,1.73,0.346,1.473988,0.000000,0.0,0.000000,0.24,0.048,34.166667,0.342507,0.3,-0.042507
13149,2024-03-07,Europa Euroliga,15:00,Monaco,Crvena zvezda,1.21,4.67,159.5,1.91,1.91,-10.5,2.00,10.50,tbCVBoAA,0.826446,0.214133,0.523560,0.523560,0.040579,0.4,0.4,NaN,NaN,150.850,32.610363,0.216177,133.490,36.923555,0.276602,144.826,145.018,114.76,180.18,0.0,0.0,0.0,0.0,0.832173,0.000000,0.961665,3.11,0.622,0.337621,0.000000,0.0,0.000000,-2.65,-0.530,-6.924528,0.590571,0.5,-0.090571
13150,2024-03-07,Europa Euroliga,16:30,Valencia,Barcelona,2.18,1.73,157.5,1.83,2.00,1.5,2.00,1.56,SEEs9qfT,0.458716,0.578035,0.546448,0.500000,0.036750,0.8,0.4,NaN,NaN,217.986,122.130859,0.560269,119.284,21.146012,0.177275,136.810,144.102,126.84,153.00,0.0,0.0,0.0,0.0,0.162761,0.062772,0.174790,-0.02,-0.004,-295.000000,0.000000,0.0,0.000000,-1.69,-0.338,-2.159763,0.561449,0.3,-0.261449
13151,2024-03-07,China Wcba Feminina,08:30,Shaanxi Tianze F,Fujian F,2.05,1.70,141.5,1.85,1.85,1.5,1.88,1.58,I94xvDSu,0.487805,0.588235,0.540541,0.540541,0.076040,0.2,0.0,NaN,NaN,274.586,249.118017,0.907250,398.470,514.511658,1.291218,284.422,209.862,137.97,85.41,0.0,0.0,0.0,0.0,0.131993,0.000000,0.12262

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Golden State Warriors,Milwaukee Bucks,226.5,1.85,1.0000,Over
1,00:30,Eua Nba,Los Angeles Lakers,Sacramento Kings,238.5,1.85,1.0000,Over
2,21:30,Eua Nba,Dallas Mavericks,Miami Heat,227.5,1.82,1.0000,Over
3,14:30,Europa Euroliga,Anadolu Efes,Bayern,165.5,1.91,1.0000,Over
4,15:00,Europa Euroliga,Monaco,Crvena zvezda,159.5,1.91,1.0000,Over
5,08:30,China Wcba Feminina,Xiamen F,Henan F,147.5,1.80,0.8964,Over
6,00:00,Eua Ncaa,Siu Edwardsville,Eastern Illinois,142.5,1.80,1.0000,Over
7,01:59,Eua Ncaa,Hawaii,CS Northridge,145.5,1.80,1.0000,Over
8,13:30,Lituânia Lkl,Nevezis,Mazeikiai,173.5,1.87,1.0000,Over
9,00:15,México Cibacopa,Frayles de Guasave,Rayos de Hermosillo,172.5,1.85,1.0000,Over
